In [ ]:
from PIL import Image  
# Open the image file  
image = Image.open('Sample.png')  
# Get the image format  
image_format = Image. format  
# Get image size  
image_size = Image. size  
# Get image mode  
image_mode = Image. mode  
# Close the image file  
image.close()  
# Print the details  
print(image_format)  
print(image_size)  
print(image_mode)  

In [50]:
import numpy as np
from scipy.signal import lfilter  # this for 1d coeff

t = [1,2,3,4,5,6,7,8,9]
coeffs = [1,2,3]
# converting the list to a numpy array

array_t = np.array(t)

sample = array_t.astype(np.int32)

# sample = sample.astype(np.float32)
print(len(sample))


sw_fir_output = lfilter(coeffs,70e3,sample)
sw_fir_output

9


array([1.42857143e-05, 5.71428571e-05, 1.42857143e-04, 2.28571429e-04,
       3.14285714e-04, 4.00000000e-04, 4.85714286e-04, 5.71428571e-04,
       6.57142857e-04])

In [32]:
import numpy as np
from scipy.signal import convolve2d

# Define a 2D input array
t = [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]]

# Define a 2D filter (coefficients)
coeffs = [[1, 2],
          [3, 4]]

# Convert the list to a NumPy array and cast to float32
sample = np.array(t).astype(np.float32)

# Apply the 2D convolution
sw_fir_output = convolve2d(sample, coeffs, mode='same')

# Output the filtered result
print("Filtered output:\n", sw_fir_output)

Filtered output:
 [[ 1.  4.  7.]
 [ 7. 23. 33.]
 [19. 53. 63.]]


In [55]:
from pynq import Overlay
import pynq.lib.dma

# Load the overlay
overlay = Overlay('./fir.bit')

dma = overlay.filter.fir_dma  # accessing the DMA IP core -> fir_dma is the name of the AXI DMA

In [56]:
from pynq import allocate

import numpy as np

t = [1,2,3,4,5,6,7,8,9]
n = len(t)

array_t = np.array(t)

sample = array_t.astype(np.int32)

In [57]:
with allocate(shape=(n,), dtype=np.int32) as in_buffer, \
     allocate(shape=(n,), dtype=np.int32) as out_buffer:
    
    np.copyto(in_buffer, sample)
    
    import time
    start_time = time.time()
    
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    stop_time = time.time()
    
    diff = stop_time - start_time
    print("time ", diff)
    print(in_buffer)
    print(out_buffer)
    in_buffer.close()
    out_buffer.close()

time  0.001598358154296875
[1 2 3 4 5 6 7 8 9]
[0 1 1 2 3 4 4 5 6]
